In [ ]:
import pandas as pd

In [ ]:
# df = pd.read_csv('../data/sales.csv')
df = pd.read_csv('sales.csv')
df.dropna(subset=['price'], inplace=True)

In [ ]:
df.head()

,cost,price,weight,purchase_date,product_type,product_level,maker,ingredient,height,width,depth
0,$333k,"$300,492",3 Ton 90 Kg,Dec 19 2008,"Q,B",advanced,M14122,"IN732052,IN732053",2.76 meters,97 cm,26 cm
1,NaN,"$430,570",3 Ton 30 Kg,Sep 10 1997,"J,D",basic,NaN,"IN732054,IN732055,IN732056,IN732057,IN732058",2.67 meters,98 cm,26 cm
2,$270k,"$213,070",3 Ton 40 Kg,Sep 05 2001,"J,D",basic,NaN,"IN732054,IN732059,IN732060",3.0 meters,93 cm,24 cm
3,NaN,"$229,174",3 Ton 50 Kg,Dec 23 2016,U,advanced,M14123,"IN732061,IN732062,IN732063",2.5 meters,102 cm,27 cm
4,$97k,"$122,659",2 Ton 970 Kg,Jan 12 2000,"D,R",advanced,NaN,"IN732064,IN732065,IN732066",2.47 meters,101 cm,26 cm


In [ ]:
df['year'] = pd.to_datetime(df.purchase_date).dt.year
train_raw = df[df.year < 2015]
test_raw = df[df.year >= 2015]

# 1. Model building

<font color='red'>Assignment:</font> Get **luxury** (**price** higher than 500k dollars) as targets for training and testing and assign them to variables **y_train** and **y_test**, respectively.

In [ ]:
# train_raw['price'] = train_raw['price'].map(lambda x: x if type(x) is float else float(x.strip('$').replace(',', '')))
# test_raw['price'] = test_raw['price'].map(lambda x: x if type(x) is float else float(x.strip('$').replace(',', '')))

In [ ]:
# train_raw['luxury'] = ''
# test_raw['luxury'] = ''
# train_raw['luxury'].values[:] = 0
# test_raw['luxury'].values[:] = 0
# train_raw['luxury'][train_raw[train_raw.price > 500000].index] = 1
# test_raw['luxury'][test_raw[test_raw.price > 500000].index] = 1

In [ ]:
# y_train = train_raw.luxury
# y_test = test_raw.luxury

In [ ]:
# Train and test data

features = list(df.columns)
target = ["price", "luxury"]
features = [fea for fea in features if fea not in target]

X_train = train_raw[features]
X_test = test_raw[features]

y_train = train_raw["price"].map(lambda x: 1 if float(x.strip("$").replace(",", "")) > 500000 else 0)
y_test = test_raw["price"].map(lambda x: 1 if float(x.strip("$").replace(",", "")) > 500000 else 0)

<font color='red'>Assignment:</font> Build a transformation class to extract, preprocess, and transform **cost**; wrap up this class with **MinMaxScaler** and **LogisticRegression** to build a classification pipeline.

In [ ]:
class Cost_Transformer(object):
    
    def fit(self, X, y=None):
        df = pd.DataFrame()
        df['cost'] = X.cost.map(self.cost2num)
        self.mean = df.mean()
        
    def transform(self, X, y=None):
        df = pd.DataFrame()
        df['cost'] = X.cost.map(self.cost2num)
        return df.fillna(self.mean)
    
    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)

    def cost2num(self, x):
        if type(x) == str: 
            x = x.strip('$').strip('k')
            return float(x)*1000
        else:
            return x

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [ ]:
steps = [('ctf', Cost_Transformer()),
         ('rescale', MinMaxScaler()),
         ('logr', LogisticRegression())]
model = Pipeline(steps)
model = model.fit(train_raw, y_train)

#### <font color='red'>Note:</font> is rescale necessary here?

<font color='red'>Question:</font> What is logistic regression? Is logistic regression a regression algorithm?

### Logistic Regression is used when the dependent variable(target) is categorical. It measures the relationship between the categorical dependent variable and one or more independent variables by estimating probabilities using a logistic/sigmoid function. Logistic regression is indeed a regression algorithm, it becomes classification when added a decision rule.

<font color='red'>Question:</font> Is logistic regression a linear algorithm? What is the relationship between logistic regression and linear regression?

### Logistic regression is a linear method, but the predictions are transformed using the logistic function. Formulas for the two methods are:
### (1) Linear: $y = a_{0} + a_{1}x + ...$
### (2) Logistic: $p = \frac{e^{b_{0} + b_{1}x}}{1 + e^{b_{0} + b_{1}x}}$

<font color='red'>Question:</font> What is sigmoid function? What is it for?

### Sigmoid funciton has the shape of an 'S', its formula is: $\frac{1}{1 + e^{-x}}$. The sigmoid function is used mostly used in classification type problem since we need to scale the data in some given specific range with a threshold.

# 2. Predicting

## 2.1 Prediction with default settings

<font color='red'>Assignment:</font> Use the **predict** function of the model to make predictions for the training set and test set, and assign the outputs to **y_train_pred** and **y_test_pred**, respectively.

In [ ]:
y_train_pred = model.predict(train_raw)

In [ ]:
y_test_pred = model.predict(test_raw)

<font color='red'>Question:</font> What are the shapes of **y_train_pred** and **y_test_pred**? What do the values in **y_train_pred** and **y_test_pred** mean?

In [ ]:
y_train_pred.shape

(2757,)

In [ ]:
y_test_pred.shape

(429,)

In [ ]:
y_test_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### 1 means we predict that this is a luxury product (price > 500K), 0 means otherwise.

<font color='red'>Assignment:</font> Calculate the training and testing accuracy scores.

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_train, y_train_pred)

0.9397896264055132

In [ ]:
accuracy_score(y_test, y_test_pred)

0.9067599067599068

<font color='red'>Assignment:</font> Calculate the precision, recall, and f1 scores. You can use classification report.

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_train, y_train_pred, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.94      0.99      0.97      2573
           1       0.70      0.17      0.27       184

    accuracy                           0.94      2757
   macro avg       0.82      0.58      0.62      2757
weighted avg       0.93      0.94      0.92      2757



In [ ]:
print(classification_report(y_test, y_test_pred, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95       383
           1       0.71      0.22      0.33        46

    accuracy                           0.91       429
   macro avg       0.81      0.60      0.64       429
weighted avg       0.89      0.91      0.88       429



<font color='red'>Assignment:</font> Get the confusion matrix.

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_train, y_train_pred)

array([[2560,   13],
       [ 153,   31]], dtype=int64)

In [ ]:
confusion_matrix(y_test, y_test_pred)

array([[379,   4],
       [ 36,  10]], dtype=int64)

#### <font color='red'>Note:</font> prediction is biased to the majority class due to data imbalance !!!

<font color='red'>Question:</font> Is this model better than the all-positive and all-negative models?

### I think this model is better than the naive models.

## 2.2 Prediction with balanced class weights

<font color='red'>Assignment:</font> Set class_weight in Logistic Regression and retrain the model.

In [ ]:
steps = [('ctf', Cost_Transformer()),
         ('rescale', MinMaxScaler()),
         ('logr', LogisticRegression(class_weight = 'balanced'))]
model = Pipeline(steps)
model = model.fit(train_raw, y_train)

In [ ]:
y_train_pred = model.predict(train_raw)
y_test_pred = model.predict(test_raw)

<font color='red'>Assignment:</font> Re-calcualte all the above metrics and confusion matrix.

In [ ]:
accuracy_score(y_train, y_train_pred)

0.8581791802684077

In [ ]:
accuracy_score(y_test, y_test_pred)

0.8717948717948718

In [ ]:
print(classification_report(y_train, y_train_pred, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.98      0.87      0.92      2573
           1       0.28      0.71      0.40       184

    accuracy                           0.86      2757
   macro avg       0.63      0.79      0.66      2757
weighted avg       0.93      0.86      0.88      2757



In [ ]:
print(classification_report(y_test, y_test_pred, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.97      0.89      0.92       383
           1       0.44      0.76      0.56        46

    accuracy                           0.87       429
   macro avg       0.71      0.82      0.74       429
weighted avg       0.91      0.87      0.89       429



In [ ]:
confusion_matrix(y_train, y_train_pred)

array([[2236,  337],
       [  54,  130]], dtype=int64)

In [ ]:
confusion_matrix(y_test, y_test_pred)

array([[339,  44],
       [ 11,  35]], dtype=int64)

#### <font color='red'>Note:</font> now prediction is more aligned with truth

<font color='red'>Question:</font> Do you see any difference? Why is the difference?

### After we balance the data class distribution, the accuracy and precision decreased, which is to be expected. The recall increased because of the decreasing in fn. The new confusion matrix really shows that the data is much more balanced now therefore the classification model works in a more reasonable way.

<font color='red'>Question:</font> Which results are more reasonable? Why?

### The balanced result is more reasonable. Standard classifier algorithms like Decision Tree and Logistic Regression have a bias towards classes which have small number of instances. They tend to only predict the majority class data. The features of the minority class are treated as noise and are often ignored. Thus, there is a high probability of misclassification of the minority class as compared to the majority class. Therefore, we need to balance the data class distribution first before building any model.

<font color='red'>Question:</font> What are the other methods to deal with imbalanced data?

### (1) Random Up-sampling and down-sampling the training data
### (2) Cluster-based Up-sampling
### (3) Modifying existing classification algorithms to make them appropriate for imbalanced data sets. The approach involves constructing several two stage classifiers from the original data and then aggregate their predictions. (Bagging based and Boosting based)